In [ ]:
!pip install -q gradio>=4.0.0 transformers>=4.35.0 torch sentencepiece accelerate protobuf>=3.20.0

In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM # GPT2 기반 모델이므로 CausalLM 사용
import warnings

warnings.filterwarnings('ignore')

# --- 모델 설정 ---
# 팀에서 파인튜닝하여 Hugging Face에 업로드한 모델 사용
MODEL_NAME = "91veMe4Plus-Project/hyunwoongko-kobart-25k" # 팀에서 파인튜닝한 모델

tokenizer = None
model = None
device = None

print("환경 설정 및 모델 로딩을 시작합니다...")

try:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")

    print(f"토크나이저 ({MODEL_NAME}) 로딩 중...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # pad_token 설정 (GPT2 기반 모델은 기본적으로 pad_token이 없음)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"모델 ({MODEL_NAME}) 로딩 중...")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME) # CausalLM으로 변경
    model.to(device) # 모델을 해당 디바이스로 이동
    model.eval() # 추론 모드로 설정
    
    print(f"모델 및 토크나이저 로딩 완료. Device: {device}")

except Exception as e:
    print(f"모델 로딩 중 심각한 오류 발생: {e}")
    print("Gradio 앱이 정상적으로 작동하지 않을 수 있습니다. 모델 이름 및 인터넷 연결을 확인해주세요.")
    # 오류 발생 시 model이나 tokenizer가 None으로 유지되어 아래 함수에서 처리됩니다.

def deobfuscate_interface(obfuscated_text):
    """
    팀에서 파인튜닝한 hyunwoongko-kobart-25k 모델을 사용하여 텍스트를 비난독화합니다.
    """
    if not model or not tokenizer:
        return "오류: 모델 또는 토크나이저가 로드되지 않았습니다. Colab 셀 실행 로그를 확인해주세요."

    if not obfuscated_text.strip():
        return "난독화된 텍스트를 입력해주세요."

    # GPT2 기반 Causal LM을 위한 프롬프트 형식
    # 파인튜닝 방식에 따라 프롬프트를 조정할 수 있습니다.
    prompt = f"난독화된 텍스트: {obfuscated_text}\n복원된 텍스트:"
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

        with torch.no_grad():
            # Causal LM을 위한 생성 파라미터
            outputs = model.generate(
                inputs['input_ids'],
                max_new_tokens=128, # 새로 생성할 토큰 수
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                top_k=50,
                repetition_penalty=1.1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                early_stopping=True
            )
        
        # 생성된 전체 텍스트에서 입력 프롬프트 부분 제거
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 프롬프트 부분을 제거하여 생성된 부분만 추출
        if "복원된 텍스트:" in generated_text:
            response = generated_text.split("복원된 텍스트:")[1].strip()
        else:
            # 프롬프트 길이만큼 제거
            prompt_length = len(tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True))
            response = generated_text[prompt_length:].strip()
        
        return response

    except Exception as e:
        print(f"텍스트 생성 중 오류 발생: {e}")
        return f"오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정
iface = gr.Interface(
    fn=deobfuscate_interface,
    inputs=gr.Textbox(
        label="난독화된 한국어 텍스트",
        placeholder="난독화된 텍스트를 입력하세요...",
        lines=5
    ),
    outputs=gr.Textbox(
        label="복원된 원본 텍스트",
        lines=5
    ),
    title="hyunwoongko-kobart-25k 한국어 텍스트 비난독화", # 모델 이름으로 제목 수정
    description="팀에서 파인튜닝한 `hyunwoongko-kobart-25k` 모델을 사용하여 난독화된 한국어 텍스트를 원본 텍스트로 복원합니다.", # 모델 이름으로 설명 수정
    examples=[ # 예제는 동일한 작업이므로 유지
        ["안녀하쎼요, 반갑쏘니댜!"],
        ["오늬 날씨갸 맆이 좆네욘."],
        ["한큿어 쳬연어 처륄예 댕햔 연귝을 해보갰습닏댜."]
    ]
)

if __name__ == '__main__':
    print("Gradio 인터페이스를 시작합니다...")
    # share=True로 설정하여 외부 접속 링크 생성
    iface.launch(debug=True, share=True)